In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from nltk.corpus import stopwords

In [3]:
stop_words = stopwords.words('spanish')

In [4]:
pd.options.display.max_rows = 30

In [5]:
# df_test = pd.read_csv('input/test.csv.gz', compression='gzip')
# df_test.columns

In [6]:
df_timing = pd.read_csv('input/train.csv', usecols=['Semana','Canal_ID'])

In [7]:
df_producto = pd.read_csv('input/producto_tabla.csv.gz', compression='gzip')
df_townstate = pd.read_csv('input/town_state.csv.gz', compression='gzip')

In [8]:
df_producto['short_names'] = df_producto.NombreProducto.str.extract('^(\D*)')

In [9]:
df_producto['short_names_processed'] = df_producto['short_names'].map(lambda x: " ".join([i for i in x.lower().split() if i not in stopwords.words('spanish')]))

In [10]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("spanish")
print(stemmer.stem("Tortillas"))

tortill


In [11]:
df_producto['short_names_processed'] = (df_producto['short_names_processed']
                                        .map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))



In [12]:
df_aux_producto = df_producto['short_names_processed'].value_counts()[df_producto['short_names_processed'].value_counts() > 10].reset_index()

In [13]:
df_producto = df_producto.ix[df_producto.short_names_processed.isin(df_aux_producto['index'])]

In [14]:
df_producto['NombreProducto'] = df_producto['NombreProducto'].str.upper()
df_producto['NombreProducto'] = df_producto.apply(lambda x :x['NombreProducto'].replace(str(x['Producto_ID']),'').strip(), axis=1 )
df_producto = pd.DataFrame(df_producto.loc[1:])
df_producto['fabricante'] = df_producto.NombreProducto.str.split(' ').str[-1]
df_producto.ix[df_producto.fabricante == '1KG','fabricante'] = ''
df_producto['NombreProducto'] = df_producto.apply(lambda x :x['NombreProducto'].replace(x['fabricante'],'').strip(), axis=1 )

reg = '(\d{1,4}G|\d{1,4}KG|\d{1,4}ML|\d{1,4}\ ML)'
df_producto['volume'] = df_producto.NombreProducto.str.extract(reg)

df_producto['NombreProducto'] = df_producto.apply(lambda x :x['NombreProducto'].replace(str(x['volume']),'').replace('  ',' ').strip(), axis=1 )

reg = '(\d{1,5}P)'
df_producto['unidade'] = df_producto.NombreProducto.str.extract(reg)
df_producto['NombreProducto'] = df_producto.NombreProducto.str.replace(reg, '').str.replace('  ',' ').str.strip()

aux = pd.DataFrame({'Producto_ID':[0],'NombreProducto':['NO IDENTIFICADO']})
df_producto = df_producto.append(aux)

df_producto['unidade'] = df_producto.volume.str.extract('([A-Z])')
df_producto['volume'] = df_producto.volume.str.replace('([A-Z])','')

# df_producto

In [15]:
df_townstate.ix[df_townstate.State.str.contains("ESTADO DE MÉXICO"), 'State'] = 'MÉXICO, D.F.'
df_townstate['Ag'] = df_townstate.Town.str.contains('AG.') * 1

In [16]:
def count_entregas(df, by):
    df['entregas_'+by] = 1
    aux = df.groupby([by]).agg({"entregas_"+by:np.size}).reset_index()
    df.drop('entregas_'+by, axis=1, inplace=True)
    return df.merge(aux, how='left', on=by)

In [17]:
def process_df(df):
    df = pd.DataFrame(df.ix[df.Demanda_uni_equil < 51])
    df = df.merge(df_producto, how='left',on='Producto_ID')
    df = df.merge(df_townstate[['Agencia_ID','Ag']], on='Agencia_ID')
#     df = df.drop(['Agencia_ID','Dev_uni_proxima','Dev_proxima','Venta_hoy','Venta_uni_hoy'], axis=1)
    df.volume = df.volume.fillna(0).astype('int')
#     df_fabricante = df[['fabricante']]
#     df_fabricante = df_fabricante.groupby('fabricante').count().reset_index()
#     df_fabricante['index_fabricante'] = df_fabricante.index
#     df = df.merge(df_fabricante, on='fabricante')
    df.shape
    df = df.ix[df.volume<500]
    
    df = df.ix[df.Demanda_uni_equil<20]
    
    df = df.ix[df.Producto_ID != 0]
    
    
    df = count_entregas(df, 'Producto_ID')
#     df = count_entregas(df, 'short_names_processed')
#     df = count_entregas(df, 'Cliente_ID')
#     df = count_entregas(df, 'fabricante')
#     df = count_entregas(df, 'Agencia_ID')
#     df = count_entregas(df, 'Canal_ID')
#     df = count_entregas(df, 'Ruta_SAK')  

    df['Cliente_ID'] = df['Cliente_ID'].astype('str')
    df['Agencia_ID'] = df['Agencia_ID'].astype('str')
    df['Canal_ID'] = df['Canal_ID'].astype('str')
    df['Ruta_SAK'] = df['Ruta_SAK'].astype('str')
    
    df.ix[df['volume']==0,'volume'] = df['volume'].median()
#     df.ix[df['entregas_fabricante'].isnull(),'entregas_fabricante'] = df['entregas_fabricante'].median()
#     df.ix[df['entregas_short_names_processed'].isnull(),'entregas_short_names_processed'] = df['entregas_short_names_processed'].median()
    
    aux = df[cols+['Demanda_uni_equil']].drop_duplicates()
        
    y = aux['Demanda_uni_equil'].values
    x = pd.get_dummies(aux[cols]).values
    return x, y, df

In [18]:
# df_process['fabricante'].value_counts()

In [19]:
cols = [
    'volume',
    'entregas_Producto_ID',
    'Cliente_ID',
#     'entregas_fabricante',
    'Agencia_ID',
    'Ruta_SAK',
    'Canal_ID',
    'short_names_processed',
#     'fabricante',
]

In [20]:
# df_process.columns

In [21]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [22]:
# rf = ExtraTreesRegressor(n_estimators=50, max_features=1.0,min_samples_split=3,n_jobs=4)

rf = ExtraTreesRegressor(n_jobs=4)

def train(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1729)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)
    print(mean_squared_error(y_test, y_pred))
        

In [23]:
# print(mean_squared_error(y[train], y_pred))

In [ ]:
# df_process

In [ ]:
for df in pd.read_csv('input/train_week3.csv', chunksize=1e5):
    x,y,df_process = process_df(df)
    train(x,y)

In [ ]:
12.33

In [ ]:
# df_process['short_names_processed']

In [ ]:
# pd.get_dummies(df_process[0:100][cols])

In [ ]:
a = pd.get_dummies(df_process[cols])

In [ ]:
rf.feature_importances_.shape

In [ ]:
a.columns

In [ ]:
plt.bar(range(0,18), height=rf.feature_importances_)

In [ ]:
f = plt.subplots()
i = 444
for v in x.T:
    i+=1
    f.hist(v)

In [ ]:
df.columns

In [ ]:
count_entregas(df, 'Agencia_ID')